In [1]:
import os
from glob import glob
import datetime
import pandas as pd
import csv
import plotly as plt
#import missingno

In [2]:
os.chdir(r'C:\Users\tailine\Desktop\Instituto de Saude e Sustentabilidade\Monitoramento acre\Dados Air_Purple')
dt=pd.read_csv('Primary_Real_Time_outside.csv').iloc[:,1:]
dt.columns= ['data', 'id', 'PM1_CF1','PM25_CF1', 'PM10_CF1', 'UptimeMinutes', 'RSSI_dbm',
          'Temp_F', 'Hum', 'PM25_ATM', 'estado', 'estacao','local', 'tipo', 't_inicial', 't_final']
dt=dt[['data', 'id','PM25_CF1','PM25_ATM', 'estado', 'estacao','local', 'tipo', 't_inicial', 't_final']]

In [3]:
dt.data=dt.data.str[:-6]+'00'

In [4]:
dt.drop_duplicates(subset=['data','estado','estacao','local'],inplace=True)

In [5]:
dt['data'] = pd.to_datetime(dt['data'], format='%Y-%m-%d %H:%M:%S')
dt=dt.set_index('data')

In [6]:
dt2=dt.groupby(by=['estado', 'estacao', 'local', 'tipo','t_inicial', 't_final']).resample('2min')[('PM25_CF1', 'PM25_ATM')].mean()
dt2.reset_index(inplace=True)

In [7]:
dt2.PM25_CF1.isnull().sum()

5676089

In [8]:
dt2.shape

(10761676, 9)

In [9]:
df_null=dt2.PM25_CF1.isnull().groupby([dt2['estado'],dt2['estacao'],dt2['local']]).sum().astype(int).reset_index(name='count')

In [10]:
df_null=df_null.merge(dt2.groupby(['estado','estacao','local'],as_index=False)['tipo'].count())
df_null['null_pct']=100*(df_null['count']/df_null.tipo)
df_null['data']=df_null.tipo-df_null['count']
df_null['data_pct']=100*(df_null['data']/df_null.tipo)

df_null.to_csv('Tabela_minuto_Primary_Real_Time_outside.csv')
#dt2[(dt2.estacao=='MPAP_AAP_01_promotoria (outside)')]

In [11]:
dt2['flg_null_H']=0
dt2.flg_null_H[dt2.PM25_CF1.isnull()]=1
dt2=dt2.set_index('data')

C:\Users\Tailine\AppData\Local\Temp\ipykernel_13960\1078889328.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt2.flg_null_H[dt2.PM25_CF1.isnull()]=1


In [12]:
df_h=dt2.groupby(by=['estado', 'estacao', 'local', 'tipo','t_inicial', 't_final']).resample('1H').agg({'PM25_CF1':'mean', 'PM25_ATM':'mean','flg_null_H':['count','sum']})

In [13]:
df_h.columns=['PM25_CF1',
            'PM25_ATM',
            'max',
            'count_null_H']

#df_h.reset_index(inplace=True)

In [14]:
df_h['null_pct_H']=100*(df_h['count_null_H']/df_h['max'])
#df_h['data']=df_h['max']-df_h['count_null_H']
#df_h['data_pct']=100*(df_h['data']/df_h['max'])

In [15]:
df_h=df_h[['PM25_CF1','PM25_ATM','max','count_null_H','null_pct_H']].reset_index()

In [16]:
df_h.head()

,estado,estacao,local,tipo,t_inicial,t_final,data,PM25_CF1,PM25_ATM,max,count_null_H,null_pct_H
0,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 19:00:00,272.072727,181.129091,24,2,8.333333
1,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 20:00:00,134.282000,92.140333,30,0,0.000000
2,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 21:00:00,12.382222,12.382222,30,21,70.000000
3,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 22:00:00,NaN,NaN,30,30,100.000000
4,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 23:00:00,NaN,NaN,30,30,100.000000


In [17]:
df_h_tratado=df_h.copy()

In [18]:
df_h_tratado.PM25_CF1[df_h_tratado.null_pct_H>=25]=None
df_h_tratado.PM25_ATM[df_h_tratado.null_pct_H>=25]=None

C:\Users\Tailine\AppData\Local\Temp\ipykernel_13960\2201963115.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h_tratado.PM25_CF1[df_h_tratado.null_pct_H>=25]=None
C:\Users\Tailine\AppData\Local\Temp\ipykernel_13960\2201963115.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_h_tratado.PM25_ATM[df_h_tratado.null_pct_H>=25]=None


In [19]:
df_h_tratado.head()

,estado,estacao,local,tipo,t_inicial,t_final,data,PM25_CF1,PM25_ATM,max,count_null_H,null_pct_H
0,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 19:00:00,272.072727,181.129091,24,2,8.333333
1,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 20:00:00,134.282000,92.140333,30,0,0.000000
2,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 21:00:00,NaN,NaN,30,21,70.000000
3,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 22:00:00,NaN,NaN,30,30,100.000000
4,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 23:00:00,NaN,NaN,30,30,100.000000


In [20]:
df_null=df_h_tratado.PM25_CF1.isnull().groupby([df_h_tratado['estacao'],df_h_tratado['local']]).sum().astype(int).reset_index(name='count')

In [21]:
df_null=df_null.merge(df_h_tratado.groupby(['estacao','local'],as_index=False)['tipo'].count())

In [22]:
df_null['null_pct']=100*(df_null['count']/df_null.tipo)
df_null['data']=df_null['tipo']-df_null['count']
df_null['data_pct']=100*(df_null['data']/df_null['tipo'])

In [23]:
df_null

,estacao,local,count,tipo,null_pct,data,data_pct
0,Afukuri AQ (outside),(-12.51475 -53.00414),6746,6772,99.616066,26,0.383934
1,FAS-Ar (outside),(-3.074871 -60.008673),3021,5371,56.246509,2350,43.753491
2,ICMBio - PARNA Viruá (outside),(1.489956 -61.002965),5096,5173,98.511502,77,1.488498
3,INPA Roraima (outside),(2.821331 -60.669727),2334,18149,12.860213,15815,87.139787
4,IPAM _ PELD-TANG (outside),(-13.059691 -52.381682),4207,6746,62.362882,2539,37.637118
5,Jaguar Ecological Reserve (outside),(-17.101682 -56.945447),10484,10995,95.352433,511,4.647567
6,MPAM_MAO_CAOMAPHURB01 (outside),(-3.090947 -60.054341),11571,18125,63.840000,6554,36.160000
7,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),8665,10533,82.265262,1868,17.734738
8,MPAP_CAL_01_promotoria (outside),(2.492567 -50.949547),6404,10533,60.799392,4129,39.200608
9,MPAP_FGO_01_promotoria (outside),(0.860345 -51.181017),5097,10535,48.381585,5438,51.618415


In [24]:
df_h_tratado

,estado,estacao,local,tipo,t_inicial,t_final,data,PM25_CF1,PM25_ATM,max,count_null_H,null_pct_H
0,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 19:00:00,272.072727,181.129091,24,2,8.333333
1,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 20:00:00,134.282000,92.140333,30,0,0.000000
2,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 21:00:00,NaN,NaN,30,21,70.000000
3,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 22:00:00,NaN,NaN,30,30,100.000000
4,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02 23:00:00,NaN,NaN,30,30,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
358733,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-30 19:00:00,2.721667,2.721667,30,0,0.000000
358734,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-30 20:00:00,3.208333,3.208333,30,0,0.000000
358735,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-30 21:00:00,6.895667,6.895667,30,0,0.000000
358736,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-30 22:00:00,13.110333,13.109667,30,0,0.000000


In [25]:
#missingno.matrix(df_h_tratado[df_h_tratado.estacao=='MPAP_AAP_01_promotoria (outside)'])

In [26]:
df_null.to_csv('Tabela_hora_Primary_Real_Time_outside.csv')
df_h_tratado.to_csv('Dado_hora_Primary_Real_Time_outside.csv')

In [27]:
dt=df_h_tratado
dt['flg_null_D']=0
dt.flg_null_D[dt.PM25_CF1.isnull()]=1


C:\Users\Tailine\AppData\Local\Temp\ipykernel_13960\4184496339.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dt.flg_null_D[dt.PM25_CF1.isnull()]=1


In [28]:
dt=dt.set_index('data')
df_d=dt.groupby(by=['estado', 'estacao', 'local', 'tipo','t_inicial', 't_final']).resample('1D').agg({'PM25_CF1':'mean', 'PM25_ATM':'mean','flg_null_D':['count','sum']})

In [29]:
df_d.columns=['PM25_CF1','PM25_ATM','max','count_null_D']
df_d['null_pct_D']=100*(df_d['count_null_D']/df_d['max'])

In [30]:
df_d=df_d[['PM25_CF1','PM25_ATM','max','count_null_D','null_pct_D']].reset_index()

In [31]:
df_d_tratado=df_d.copy()
df_d_tratado.PM25_CF1[df_d_tratado.null_pct_D>=33]=None
df_d_tratado.PM25_ATM[df_d_tratado.null_pct_D>=33]=None
df_d_tratado

C:\Users\Tailine\AppData\Local\Temp\ipykernel_13960\345961507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d_tratado.PM25_CF1[df_d_tratado.null_pct_D>=33]=None
C:\Users\Tailine\AppData\Local\Temp\ipykernel_13960\345961507.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_d_tratado.PM25_ATM[df_d_tratado.null_pct_D>=33]=None


,estado,estacao,local,tipo,t_inicial,t_final,data,PM25_CF1,PM25_ATM,max,count_null_D,null_pct_D
0,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-02,NaN,NaN,5,3,60.000000
1,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-03,NaN,NaN,24,24,100.000000
2,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-04,NaN,NaN,24,24,100.000000
3,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-05,NaN,NaN,24,23,95.833333
4,Amapa,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),Primary Real Time,04_02_2021,06_15_2022,2021-04-06,NaN,NaN,24,24,100.000000
...,...,...,...,...,...,...,...,...,...,...,...,...
14969,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-26,8.815576,8.779475,24,0,0.000000
14970,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-27,6.594404,6.592654,24,0,0.000000
14971,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-28,4.795566,4.793209,24,0,0.000000
14972,Tocantins,MPTO_PMW_01_procuradoriageral (outside),(-10.176555 -48.332757),Primary Real Time,06_02_2020,06_30_2022,2022-06-29,7.254865,7.253676,24,0,0.000000


In [32]:
df_null=df_d_tratado.PM25_CF1.isnull().groupby([df_d_tratado['estacao'],df_d_tratado['local']]).sum().astype(int).reset_index(name='count')
df_null=df_null.merge(df_d_tratado.groupby(['estacao','local'],as_index=False)['tipo'].count())

In [33]:
df_null

,estacao,local,count,tipo
0,Afukuri AQ (outside),(-12.51475 -53.00414),282,283
1,FAS-Ar (outside),(-3.074871 -60.008673),127,225
2,ICMBio - PARNA Viruá (outside),(1.489956 -61.002965),213,217
3,INPA Roraima (outside),(2.821331 -60.669727),116,757
4,IPAM _ PELD-TANG (outside),(-13.059691 -52.381682),171,282
5,Jaguar Ecological Reserve (outside),(-17.101682 -56.945447),444,459
6,MPAM_MAO_CAOMAPHURB01 (outside),(-3.090947 -60.054341),737,756
7,MPAP_AAP_01_promotoria (outside),(2.04586 -50.792344),365,440
8,MPAP_CAL_01_promotoria (outside),(2.492567 -50.949547),273,440
9,MPAP_FGO_01_promotoria (outside),(0.860345 -51.181017),215,440


In [34]:
df_null['null_pct']=100*(df_null['count']/df_null.tipo)
df_null['data']=df_null['tipo']-df_null['count']
df_null['data_pct']=100*(df_null['data']/df_null['tipo'])

df_null.to_csv('Tabela_dia_Primary_Real_Time_outside.csv')
df_d_tratado.to_csv('Dado_dia_Primary_Real_Time_outside.csv')

In [ ]:
df_h_tratado.info()
df_h_tratado['estacao'].nunique()
list_estacao = list(set(df_h_tratado['estacao']))
list_estado = list(df_h_tratado['estado'])

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.ticker as ticker

In [ ]:
am1=df_d_tratado[df_d_tratado.estado=='Amapa']
am2=df_h_tratado[df_h_tratado.estado=='Amapa']

In [ ]:
sns.set_style('darkgrid')
sns.set(rc={'figure.figsize':(14,8)})

ax = sns.lineplot(data=df_d_tratado, x ='data', y = 'PM25_ATM',
                  hue='estacao', palette='viridis',col = "estado",col_wrap = 2,
                  legend='full', lw=3)

ax.xaxis.set_major_locator(ticker.MultipleLocator(4))
plt.legend(bbox_to_anchor=(1, 1))
plt.ylabel('PM2.5 (µg/m3)')
plt.xlabel('Year-Month')
plt.show()

In [ ]:
g = sns.FacetGrid(df_d_tratado, col="estado", row="estacao", height=4.2, aspect=1.9)
g = g.map(sns.barplot, 'data', 'max', palette='viridis')

g.set_xticklabels(rotation = 90)
plt.show()

In [ ]:
g = sns.relplot(data = df_d_tratado, x = "data", y = "PM25_ATM",
                col = "estado", hue = "estacao",
                kind = "line", palette = "Spectral",   
                linewidth = 4, zorder = 5,
                col_wrap = 2, height = 3, aspect = 1.5, legend = 'none'
               )

#add text and silhouettes
for time, ax in g.axes_dict.items():
    ax.text(.1, .85, time,
            transform = ax.transAxes, fontweight="bold"
              )
    sns.lineplot(data = df_d_tratado, x = "data", y = "PM25_ATM", units="estacao",
                 estimator = None, color= ".7", linewidth=1, ax=ax,legend='full'
                )

ax.set_xticks('')
g.set_titles("")
g.set_axis_labels("", "PM25_ATM")
g.tight_layout()

In [ ]:
?sns.lineplot